 <div class="alert alert-block alert-info">
    <b>Note:</b> 
    Interchange is in the process of replacing ParmEd in many workflows, but it still in an alpha testing phase. Our internal tests indicate it is reliable for many small-molecule systems, but it is not yet reliable for complex, multi-component systems and there are likely still rough edges throughout. Feedback is welcome on the <a href=https://github.com/openforcefield/openff-interchange/issues/>Interchange issue tracker.</a></div>

## Using OpenFF force fields in Amber and GROMACS

The Open Forcefield Toolkit can create parametrized `openmm.System` objects that can be natively simulated with OpenMM. This example shows the Interchange project can enable parallel workflows using Amber and GROMACS.

### Preparing an OpenFF Topology

We start by loading a PDB file containing one copy of ethanol and cyclohexane. Our goal is to create an OpenFF `Topology` object describing this system that we can parametrize with the SMIRNOFF-format "Sage" force field.

The two `Molecule` objects created from the SMILES strings can contain information such as partial charges and stereochemistry that is not included in an OpenMM topology. In this example, partial charges are not explicitly given, and `ForceField` will assign AM1/BCC charges as specified by the "Sage" force field. Note that the OpenFF Toolkit produces partial charges that do not depend on the input conformation of parameterized molecules. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/latest/faq.html#the-partial-charges-generated-by-the-toolkit-don-t-seem-to-depend-on-the-molecule-s-conformation-is-this-a-bug) for more information.

In [1]:
try:
    from openmm import app
except ImportError:
    from simtk.openmm import app

from openff.toolkit.topology import FrozenMolecule, Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
import pdb
import os

In [2]:
tetra_mom = Molecule.from_file("../../../build_polymer/mom/mom_tetramer_mbuild.mol")

In [3]:
# Load the PDB file using OpenMM and save the OpenMM Topology
pdbfile = app.PDBFile("../../../build_polymer/mom/mom_tetramer_mbuild_renum.pdb")
omm_topology = pdbfile.topology
omm_topology

<Topology; 1 chains, 6 residues, 197 atoms, 208 bonds>

In [4]:
# Create the OpenFF Topology.
off_topology = Topology.from_openmm(
    omm_topology, unique_molecules=[tetra_mom]
)


In [8]:
# Modified OpenFF to increase maxAtoms for AM1BCC method to 500 atoms
if not os.path.exists('terphenyl_mom_tetramer_charges.sdf'):
    tetra_mom.assign_partial_charges(partial_charge_method="am1bcc")
    tetra_mom.to_file('terphenyl_mom_tetramer_charges.sdf', file_format='sdf')
else:
    tetra_mom = Molecule.from_file('terphenyl_mom_tetramer_charges.sdf')

### Preparing an OpenFF ForceField

Once the `ForceField` class is imported, the only decision to make is which force field to use. An exhaustive list of force fields released by the Open Force Field Initiative can be found [here](from openff.toolkit.typing.engines.smirnoff import ForceField
).

Here we will use force field from the "Sage" line.

In [9]:
forcefield = ForceField("openff-2.0.0.offxml")

### Preparing an OpenMM System

Once a force field and topology have been loaded, an `openmm.System` can be generated natively with the OpenFF Toolkit.

In [10]:
omm_system = forcefield.create_openmm_system(off_topology, charge_from_molecules = [tetra_mom])
omm_system

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7f44a8c4dc30> >

### Preparing an Interchange object

To exports to engines other than OpenMM, we will make use of the [Interchange](https://openff-interchange.readthedocs.io/) project. There is a high-level `Interchange.from_smirnoff` function that consumes OpenFF Toolkit and ForceField objects and produces an `Interchange` object which can then be exported to formats understood by other molecular simulation engines. This extra step is needed to provide a clean interface between _applied_ parameters and engines. Note also that this step does not require an OpenMM System to be generated; `ForceField.create_openmm_system` does not need to be called to use Amber and GROMACS.

In [13]:
from openff.interchange.components.interchange import Interchange

interchange = Interchange.from_smirnoff(
    force_field=forcefield,
    topology=off_topology,
    charge_from_molecules = [tetra_mom]
)
interchange.positions = pdbfile.positions
interchange

Interchange with 7 potential handlers, non-periodic topology with 197 atoms.

### Exporting to Amber and GROMACS files

Once an `Interchange` object has been constructed, its API can be used to export to files understood by GROMACS, Amber, and more.

In [14]:
# Export GROMACS files.
interchange.to_top("terphenyl_mom_tetramer.top")
interchange.to_gro("terphenyl_mom_tetramer.gro")

/home/tfobe/anaconda3/envs/terphenyl/lib/python3.9/site-packages/openff/interchange/interop/internal/gromacs.py:164: UserWarning: WARNING: System defined with no box vectors, which GROMACS does not offically support in versions 2020 or newer (see https://gitlab.com/gromacs/gromacs/-/issues/3526). Setting box vectors to a 5  nm cube.
  warnings.warn(


### Validating the conversion to Amber files

The Interchange project includes functions that take in an `Interchange` object and call out to simulation engines to run single-point energy calculations (with no minimization or dynamics) for the purpose of validating the export layer with each engine. Under the hood, each of these functions calls API points like those used above while converting to files understood by each engine. These rely on having each engine installed and accessible in the current `$PATH`.

In [15]:
from openff.interchange.drivers import get_amber_energies, get_openmm_energies

In [16]:
openmm_energies = get_openmm_energies(interchange)
openmm_energies.energies

{'Bond': 219.46983149172524 <Unit('kilojoule / mole')>,
 'Angle': 1297.1353727280873 <Unit('kilojoule / mole')>,
 'Torsion': 366.0213327091996 <Unit('kilojoule / mole')>,
 'vdW': 74921.10418843284 <Unit('kilojoule / mole')>,
 'Electrostatics': -978.416909358118 <Unit('kilojoule / mole')>}

In [17]:
!cat terphenyl_pom_tetramer.top
!cat terphenyl_pom_tetramer.gro

cat: terphenyl_pom_tetramer.top: No such file or directory
cat: terphenyl_pom_tetramer.gro: No such file or directory


### Appendix: Validating the conversion to GROMACS and LAMMPS files

If GROMACS and/or LAMMPS are installed on your machine, the same comparisons can also take place with those engines. They are available via `conda` by running a command like:

```conda install gromacs lammps -c conda-forge -c bioconda```

In [18]:
from distutils.spawn import find_executable
from pprint import pprint

from openff.interchange.drivers import get_gromacs_energies, get_lammps_energies

In [19]:
if find_executable("lmp_serial"):
    pprint(get_lammps_energies(interchange).energies)

In [20]:
if find_executable("gmx"):
    pprint(get_gromacs_energies(interchange).energies)

UnsupportedCutoffMethodError: PME is not valid with a non-periodic system.

Finally, there is a helper function `get_summary_data` that will attempt to run drivers of each engine. A summary reported is prepared as a Pandas `DataFrame`.